# Funções auxiliares

In [37]:
from sklearn.base import BaseEstimator, TransformerMixin
from datetime import datetime, date
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from collections import Counter

class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas = X.select_dtypes(include='number').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasNumericas].to_numpy() 

class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas = X.select_dtypes(exclude='number').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasCategoricas].to_numpy()

def calcularTempoAlunoCurso(entrada):
    formatoData = "%d/%m/%Y"
    dataEntrada = datetime.strptime(entrada, formatoData).date()
    today = date.today()
    return today.year - dataEntrada.year

def extrairRenda(entrada):
    if ((entrada == 'Não declarada') or (entrada == 'NÃO DECLARADA')):
        renda = 0
    else:
        if (entrada.find('>') >=0):
            
            renda = float(entrada[entrada.find('>')+1:].replace(',','.'))
        elif (entrada.find('<') >=0): 
            renda = float(entrada[entrada.find('<=')+2:].replace(',','.'))
        else:
            renda = entrada
    return renda    


class AtributosDesejados(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasIndesejadas = ['Aluno','InstituicaoEducacional','TipoLocalizacao']
        #self.colunasIndesejadas = ['Aluno','InstituicaoEducacional']
        return self
    def transform(self, X, y=None):
        Xdrop = X.drop(self.colunasIndesejadas,axis=1)
        #Xdrop['Matricula'] = Xdrop['Matricula'].apply(calcularTempoAlunoCurso)
        #Xdrop['Idade'] = Xdrop['Idade']/10
        if 'Renda' in X:
            X['Renda'] = X['Renda'].apply(extrairRenda)
        return Xdrop      

def maisFrequente(y):
    return Counter(y.flat).most_common(1)[0][0]

class ZeroR():
    def fit(self, X, y):
        self.resposta = maisFrequente(y)
    def predict(self, X):
        y = np.empty((X.shape[0]))
        y[:] = self.resposta
        return y

In [38]:
import pandas as pd
import math


def truncate(number, decimals=0):
    """
    Returns a value truncated to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer.")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more.")
    elif decimals == 0:
        return math.trunc(number)

    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor

# Carregando dados atuais

In [39]:
import numpy as np
import pandas as pd

#Parametros
arquivoBaseDadosAtuais = './dados/pnp2021/RepositorioPnp2021-ifes.csv'
separadorCsvBaseDadosAtuais=';'
arquivoSituacoesEvasaoBase='./dados/pnp2021/situacoes.csv'
separadorCsvSituacaoesEvasao=';'
arquivoResultado = './dados/dadosEvasao'
separadorCsvArquivoResultado=';'

#Ler situações que são consideradas evasão
situacoes_evasao = pd.read_csv(arquivoSituacoesEvasaoBase,separadorCsvSituacaoesEvasao)
situacoes_evasao=situacoes_evasao['SituacaoEvasao'].to_numpy()

#Ler Base de dados
dfAtual=pd.read_csv(arquivoBaseDadosAtuais,separadorCsvBaseDadosAtuais)

#dfAtual = dfAtual.apply(lambda x: x.astype(str).str.upper())

dfAtualCopia = dfAtual

y_atual = dfAtual['SituacaoMatricula']

y_atual_binary = np.where(np.isin(y_atual,situacoes_evasao),1,0 )



print(np.count_nonzero(y_atual_binary==1))


2411


In [40]:
import pickle
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

  
models = {'ZeroR','Decision Trees','KNN','Random Forest','Extra Trees','Ada Boost','Bagging','Gradient Boosting'}
accuracy, precision, recall, f1 = {}, {}, {},{}

for model in models:
    X_atual = dfAtual.drop(['SituacaoMatricula'],axis=1)
    X_atual2 = dfAtual.drop(['SituacaoMatricula'],axis=1)
    nomeModelo=model+'.pkl'
    pickled_model = pickle.load(open(nomeModelo, 'rb'))
    y_predict = pickled_model.predict(X_atual)

    cm = confusion_matrix(y_atual_binary, y_predict)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['Não Evadiu','Evadiu'])
                            
    disp.plot()
    disp.ax_.set(xlabel='Predição', ylabel='Valor Real',title="Matriz de Confusão para o algoritmo "+ model) 

    if(model !='ZeroR'):
        proba = pickled_model.predict_proba(X_atual2)
        dfAtualCopia['Predicao'] = y_predict
        dfAtualCopia['ProbabilidadeEvasao'] = proba[0:,1:].round(decimals=2)

        dfAtualCopia.to_csv(arquivoResultado+model+'.csv',sep = separadorCsvBaseDadosAtuais, index = False)

    accuracy[model] = accuracy_score(y_atual_binary,y_predict)
    precision[model] = precision_score(y_atual_binary,y_predict)
    f1[model] = f1_score(y_atual_binary,y_predict)
    recall[model] = recall_score(y_atual_binary,y_predict)

df_model = pd.DataFrame(index=models, columns=['Accuracy', 'Precision','Recall','F1-Score'])
df_model['Accuracy'] = accuracy.values()
df_model['Precision'] = precision.values()
df_model['F1-Score'] = f1.values()
df_model['Recall'] = recall.values()

print('Tabela comparando as métricas de avaliação')
df_model.style.highlight_max(color = 'lightgreen', axis = 0)






KeyError: "['GrauAcademico'] not in index"